In [15]:
### Import packages for preprocessing

import pandas as pd
import string
import spacy

## Import packages to create absolute file path & make code independent of operating system

from pathlib import Path
import os.path


In [2]:
### Verify that spaCy was installed and that models and data were successfully downloaded

nlp = spacy.load('en_core_web_sm')

In [8]:
### CSV-data into a dataframe

base_path = Path("__file__").parent

full_path = (base_path / "../../data/raw/stackoverflow_raw.csv").resolve()

df_stackoverflow = pd.read_csv(os.path.join(full_path))

In [11]:
df_stackoverflow.head()

,Unnamed: 0,question_id,question_id_check,question_title,question_text,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,...,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,post_type_id,score,tags,view_count,answer_id,answer_text
0,0,1859864,1859864,How to create an integer array in Python?,"<p>It should not be so hard. I mean in C, </p>...",1859889.0,8,3,NaN,2009-12-07 13:08:31.840000+00:00,...,NaN,NaN,NaN,226353.0,1,33,python|arrays,244103,56431492,<pre><code>import numpy as np\n\nnew_array=np....
1,1,59920382,59920382,How can I add rows with same string value in s...,<p>i have tried to make this df to the on afte...,59920693.0,2,3,NaN,2020-01-26 16:32:52.357000+00:00,...,NaN,10035985.0,NaN,12786329.0,1,0,python|pandas,32,59920693,<p>you need <code>reset_index()</code> along w...
2,2,60142014,60142014,Custom Sort Using a Comma Separated String Bas...,<p>This is an extension to my original questio...,60142496.0,1,3,NaN,2020-02-09 22:37:36.787000+00:00,...,NaN,10018602.0,NaN,10018602.0,1,1,excel|vba,36,60142496,<p>I couldn't get the <code>Worksheet.Sort</co...
3,3,57150458,57150458,Can a Retry be configured to retry on null?,<p>I am just starting out with resilience4j an...,57151117.0,1,2,NaN,2019-07-22 16:48:52.310000+00:00,...,NaN,NaN,NaN,1997707.0,1,1,java|resilience4j,77,57151117,<p>Create a custom <code>RetryConfig</code> wi...
4,4,57066427,57066427,Sum multiple columns that have specific name i...,<p>I would like to sum the values of <code>Var...,57067383.0,4,2,NaN,2019-07-16 22:52:19.370000+00:00,...,NaN,89482.0,NaN,11725370.0,1,2,r|sum,92,57067383,<p>Another <code>dplyr</code> way is to use he...


In [12]:
df_stackoverflow.shape

(1789030, 23)

In [13]:
df_stackoverflow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1789030 entries, 0 to 1789029
Data columns (total 23 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   Unnamed: 0                int64  
 1   question_id               int64  
 2   question_id_check         int64  
 3   question_title            object 
 4   question_text             object 
 5   accepted_answer_id        float64
 6   answer_count              int64  
 7   comment_count             int64  
 8   community_owned_date      object 
 9   creation_date             object 
 10  favorite_count            float64
 11  last_activity_date        object 
 12  last_edit_date            object 
 13  last_editor_display_name  object 
 14  last_editor_user_id       float64
 15  owner_display_name        object 
 16  owner_user_id             float64
 17  post_type_id              int64  
 18  score                     int64  
 19  tags                      object 
 20  view_count              

In [14]:
df_stackoverflow.score.value_counts()

 0       841004
 1       362374
 2       140655
-1       110430
 3        62476
          ...  
 1232         1
 2730         1
 1699         1
 2266         1
 2486         1
Name: score, Length: 1510, dtype: int64

In [ ]:
### https://towardsdatascience.com/machine-learning-for-text-classification-using-spacy-in-python-b276b4051a49

### Code from Laura for preprocessing 

"""
Main script to preprocess a list of texts
"""

from utils.data_wranglers import wrangle_amazon_reviews, sample_amazon_reviews, \
    get_amazon_review
from utils.spacy_helpers import text2tokens_, text2sentences, text2sentences_custom, text2tokens

#
# read data
#

df = wrangle_amazon_reviews()
# sample_review = sample_amazon_reviews(df, 1)[0]
# review_0 = get_amazon_review(df, 0)
review_0_raw = df.loc[0, 'customer_reviews']

#
# splitting sentences
#

sentences = text2sentences(review_0_raw)
sentences_custom = text2sentences_custom(review_0_raw)

#
# tokenization
#

# tokenize a sentence
tokens = text2tokens(sentences[0])
tokens_ = text2tokens_(sentences[0])
[print(t) for t in tokens_]

#
# output
#

print('original review:', review_0_raw)
print('sentences:', sentences)
print('sentences custom:', sentences_custom)
print('tokens:', tokens)

In [ ]:
### Tokenization


In [ ]:
### Removing stopwords



In [ ]:
### Lemmatization



In [ ]:
### Part of speech tagging



In [ ]:
### Dependency parsing



In [ ]:
### Word vectorization

